# Pair Programming ANOVA

En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.

En el ejercicio de hoy tendréis que hacer un ANOVA con vuestro datos y hacer una interpretación de los resultados.

📌 NOTA Puede que vuestros datos no se ajusten o no cumplan todas las asunciones, no pasa nada, haced el ANOVA e interpretad los resultados. En próximas lecciones aprenderemos que podemos hacer cuando nos encontramos en esta situación.

In [6]:
import numpy as np
import pandas as pd
import random 

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

plt.rcParams["figure.figsize"] = (10,8) 

import sys
sys.path.append("../../")
from src import funciones as fun
from src import variables as var

In [7]:
df = pd.read_pickle('../archivos/coste_vida_estandar.pkl')
df.head()

,country,basic_boxcox,basic,mcdonalds,cappuccino,milk,rice,eggs,chicken,beef,...,public_transport_ticket,taxi,gasoline,internet,gym_monthly,cinema,preschool,primary_school,apt_3beds_outcentre,monthly_salary
0,South Korea,16.684542,182.13,-0.219373,0.601852,2.395833,1.046243,1.304348,0.747273,4.294342,...,-0.153846,-0.258103,0.245902,-0.302919,1.074934,0.208531,0.101671,1.262442,0.904358,0.517125
1,China,10.443987,66.00,-0.350427,0.625000,3.520833,-0.289017,-0.014493,-0.292727,0.338771,...,-0.408284,-0.691550,-0.131148,-0.439942,1.455932,0.053956,1.617468,2.248014,0.750069,-0.004580
2,China,9.945543,59.65,-0.552707,0.421296,1.791667,-0.398844,-0.384058,-0.490909,0.148560,...,-0.532544,-0.744625,-0.147541,-0.450326,-0.011987,0.053956,0.335639,2.036034,-0.045124,-0.090119
3,India,8.515986,43.57,-0.923077,-0.069444,-0.625000,-0.514451,-0.934783,-0.505455,-0.656716,...,-0.597633,-0.797701,0.114754,-0.635978,-0.452562,-0.564346,-0.358551,-0.390842,-0.034428,-0.324674
4,India,9.816500,58.07,-0.746439,-0.398148,-0.666667,-0.502890,-0.884058,-0.483636,-0.690038,...,-0.455621,-0.877313,-0.065574,-0.670930,-0.661335,-0.475392,-0.409781,-0.590637,-0.371835,-0.347005


Aunque sabemos que con la variable dependiente sin normalizar adecuadamente y sin cumplir las demas asunciones con nuestras variables predictoras sabemos que no podemos realizar un modelo de regresión lineal, vamos a continuar el proceso y sacar una ANOVA como actividad didáctica.

In [ ]:
# eliminamos la variable respuesta original, ya que para realizar la ANOVA utilizaremos la tratada con el método de normalización.
df.drop('basic', axis = 1, inplace=True)

In [14]:
df.columns

Index(['country', 'basic_boxcox', 'mcdonalds', 'cappuccino', 'milk', 'rice',
       'eggs', 'chicken', 'beef', 'banana', 'water', 'wine', 'beer',
       'cigarettes_marlboro', 'public_transport_ticket', 'taxi', 'gasoline',
       'internet', 'gym_monthly', 'cinema', 'preschool', 'primary_school',
       'apt_3beds_outcentre', 'monthly_salary'],
      dtype='object')

In [16]:
lm = ols('basic_boxcox ~ country + mcdonalds + cappuccino + milk + rice + eggs + chicken + beef + banana + water + wine + beer + cigarettes_marlboro + public_transport_ticket + taxi + gasoline + internet + gym_monthly + cinema + preschool + primary_school + apt_3beds_outcentre + monthly_salary', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
country,207.0,59045.380483,285.243384,65.633744,0.000000e+00
mcdonalds,1.0,142.121305,142.121305,32.701734,1.148357e-08
cappuccino,1.0,183.827615,183.827615,42.298245,8.745378e-11
milk,1.0,59.882982,59.882982,13.778915,2.082722e-04
rice,1.0,43.789549,43.789549,10.075859,1.513003e-03
eggs,1.0,129.297632,129.297632,29.751041,5.192365e-08
chicken,1.0,64.244729,64.244729,14.782541,1.224173e-04
beef,1.0,27.988319,27.988319,6.440038,1.119316e-02
banana,1.0,100.846180,100.846180,23.204438,1.507488e-06
water,1.0,5.945454,5.945454,1.368033,2.422159e-01


In [17]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           basic_boxcox   R-squared:                       0.767
Model:                            OLS   Adj. R-squared:                  0.755
Method:                 Least Squares   F-statistic:                     60.95
Date:                Fri, 19 May 2023   Prob (F-statistic):               0.00
Time:                        21:36:57   Log-Likelihood:                -9504.1
No. Observations:                4468   AIC:                         1.947e+04
Df Residuals:                    4238   BIC:                         2.094e+04
Df Model:                         229                                         
Covariance Type:            nonrobust                                         
===============================================================================================================
                                                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------
Intercept                                      10.1014      1.530      6.601      0.000       7.101      13.102
country[T.Albania]                              0.8792      1.629      0.540      0.589      -2.314       4.072
country[T.Algeria]                              0.3996      1.572      0.254      0.799      -2.683       3.482
country[T.Andorra]                              0.0784      1.956      0.040      0.968      -3.755       3.912
country[T.Angola]                              -0.9553      2.593     -0.368      0.713      -6.040       4.129
country[T.Anguilla]                             5.4693      2.605      2.099      0.036       0.362      10.577
country[T.Antigua And Barbuda]                  3.0574      2.613      1.170      0.242      -2.065       8.180
country[T.Argentina]                            0.5701      1.599      0.357      0.721      -2.564       3.705
country[T.Armenia]                              2.9887      1.754      1.704      0.088      -0.449       6.427
country[T.Aruba]                                4.8869      2.603      1.878      0.060      -0.216       9.989
country[T.Australia]                            3.6870      1.643      2.245      0.025       0.467       6.907
country[T.Austria]                              6.6064      1.666      3.966      0.000       3.340       9.872
country[T.Azerbaijan]                           0.7238      1.666      0.434      0.664      -2.542       3.990
country[T.Bahamas]                              6.6934      2.644      2.531      0.011       1.509      11.877
country[T.Bahrain]                              6.3029      1.970      3.199      0.001       2.441      10.165
country[T.Bangladesh]                          -0.7561      1.660     -0.455      0.649      -4.011       2.499
country[T.Barbados]                            -0.6299      2.620     -0.240      0.810      -5.766       4.506
country[T.Belarus]                              1.7553      1.653      1.062      0.288      -1.486       4.996
country[T.Belgium]                              3.9024      1.585      2.462      0.014       0.795       7.010
country[T.Belize]                              -0.2227      1.812     -0.123      0.902      -3.775       3.330
country[T.Benin]                               -3.3195      2.602     -1.276      0.202      -8.421       1.782
country[T.Bermuda]                              3.2987      2.670      1.235      0.217      -1.937       8.534
country[T.Bhutan]                              -1.4945      1.946     -0.768      0.443      -5.310       2.321
country[T.Bolivia]                             -0.6270      1.745     -0.359      0.719      -4.049       2.795
country[T.Bosnia And Herzegovina]               5.1809     

Todas nuestras variables explican en cierta medida nuestra variable dependiente. No obstante, debemos recordar que estos datos han sido extraidos sin cumplir con una de las asunciones báscias que es el de la normalidad de la variable dependiente por lo que se trata de un ejercicio puramente práctico.

En realidad, nuestra conclusión continua siendo que no podemos realizar una regresión linal como método adecuado para predecir nuestra variable dependiente.